# Calculate new Data

This code processes YouTube commentary data from various election cycles. First, the relevant CSV files for the years 2016, 2020 and 2024 are read in and saved in DataFrames. The data is then standardized, merged and prepared for further analysis.

# Imports

In [1]:
import pandas as pd
import csv

# Concat

In this section, first the comments and then the videos are imported and merged. The data is also saved as a new csv file.

In [2]:
# Dateipfade zu den CSV-Dateien
file_2016 = 'Export_csv/YouTube_Exporte/export_comments_2016.csv'
file_2020 = 'Export_csv/YouTube_Exporte/export_comments_2020.csv'
file_2024 = 'Export_csv/YouTube_Exporte/export_comments_2024.csv'

# Spaltennamen definieren
columns = ['custom_index', 'comment_ID', 'parent_ID', 'video_ID', 'author', 'text', 'like_count', 'published_At']

dtype_mapping = {
    'like_count': 'Int64',
}

# CSV-Dateien einlesen und in DataFrames speichern
df_2016 = pd.read_csv(file_2016, header=0, encoding='utf-8', names=columns, dtype=dtype_mapping, low_memory=False)
df_2020 = pd.read_csv(file_2020, header=0, encoding='utf-8', names=columns, dtype=dtype_mapping, low_memory=False)
df_2024 = pd.read_csv(file_2024, header=0, encoding='utf-8', names=columns, dtype=dtype_mapping, low_memory=False)  

# DataFrames zusammenführen
df_comments = pd.concat([df_2016, df_2020, df_2024], ignore_index=True)

In [3]:
df_2016.info()
df_2020.info()
df_2024.info()
df_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2825 entries, 0 to 2824
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   custom_index  2825 non-null   int64 
 1   comment_ID    2825 non-null   object
 2   parent_ID     425 non-null    object
 3   video_ID      2825 non-null   object
 4   author        2825 non-null   object
 5   text          2825 non-null   object
 6   like_count    2825 non-null   Int64 
 7   published_At  2825 non-null   object
dtypes: Int64(1), int64(1), object(6)
memory usage: 179.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53648 entries, 0 to 53647
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   custom_index  53648 non-null  int64 
 1   comment_ID    53648 non-null  object
 2   parent_ID     12082 non-null  object
 3   video_ID      53648 non-null  object
 4   author        53582 non-null  object
 5   text 

In [4]:
df_comments.columns

Index(['custom_index', 'comment_ID', 'parent_ID', 'video_ID', 'author', 'text',
       'like_count', 'published_At'],
      dtype='object')

In [5]:
df_comments.to_csv('Export_csv/YouTube_Exporte/export_comments_all.csv', encoding='utf-8', quoting=csv.QUOTE_ALL)

### Videos

In [6]:
# Dateipfade zu den CSV-Dateien
file_2016_videos = 'Export_csv/YouTube_Exporte/export_videos_2016.csv'
file_2020_videos = 'Export_csv/YouTube_Exporte/export_videos_2020.csv'
file_2024_videos = 'Export_csv/YouTube_Exporte/export_videos_2024.csv'

# Spaltennamen definieren
columns = ['custom_index', 'video_Name','video_ID','publisher','view_Count','comment_Count','published_at']

# CSV-Dateien einlesen und in DataFrames speichern
df_videos_2016 = pd.read_csv(file_2016_videos, header=0, encoding='utf-8',names=columns)
df_videos_2020 = pd.read_csv(file_2020_videos, header=0, encoding='utf-8',names=columns)
df_videos_2024 = pd.read_csv(file_2024_videos, header=0, encoding='utf-8',names=columns)  

# DataFrames zusammenführen
df_videos = pd.concat([df_videos_2016, df_videos_2020, df_videos_2024], ignore_index=True)

In [7]:
df_videos.columns

Index(['custom_index', 'video_Name', 'video_ID', 'publisher', 'view_Count',
       'comment_Count', 'published_at'],
      dtype='object')

In [8]:
df_comments.to_csv('Export_csv/YouTube_Exporte/export_videos_all.csv', encoding='utf-8', quoting=csv.QUOTE_ALL)

# Calculate new data
In this section, new columns are calculated for both df (e.g. election cycle or year of release of the video) and then saved as a new csv.

In [9]:
# Überprüfe die Spaltennamen beider DataFrames
print('Columns of df_comments:', df_comments.columns)
print('Columns of df_videos:', df_videos.columns)

Columns of df_comments: Index(['custom_index', 'comment_ID', 'parent_ID', 'video_ID', 'author', 'text',
       'like_count', 'published_At'],
      dtype='object')
Columns of df_videos: Index(['custom_index', 'video_Name', 'video_ID', 'publisher', 'view_Count',
       'comment_Count', 'published_at'],
      dtype='object')


In [10]:
df_comments['published_At'] = pd.to_datetime(df_comments['published_At'])
df_comments['year'] = df_comments['published_At'].dt.year
df_comments['comment_length'] = df_comments['text'].apply(lambda x: len(x) if isinstance(x, str) else 0)

In [11]:
# Video Jahr zu Comments hinzufügen 
df_comments['video_ID'] = df_comments['video_ID'].astype(str)
df_videos['video_ID'] = df_videos['video_ID'].astype(str)
df_videos['video_year'] = pd.to_datetime(df_videos['published_at']).dt.year

# Merge auf Basis der video_ID für die Spalte video_year
df_comments = pd.merge(df_comments, df_videos[['video_ID', 'video_year']], on='video_ID', how='left')

print(df_comments.head())

   custom_index                  comment_ID parent_ID     video_ID  \
0             1  Ugze0aXA6zTFey241M94AaABAg       NaN  SHG0ezLiVGc   
1             2  Ugwa-qc1G5P9VZEjHuF4AaABAg       NaN  SHG0ezLiVGc   
2             3  UgxOS3FTiYFkiS9bGf14AaABAg       NaN  SHG0ezLiVGc   
3             4  UgyQQqHl9-Y13pPiml54AaABAg       NaN  SHG0ezLiVGc   
4             5  UgwBHGY9omjjiF2f1Gh4AaABAg       NaN  SHG0ezLiVGc   

                   author                                               text  \
0         @dennistims2604                           2025 and still laughing.   
1            @klvd43rs565  DECADES AGO-  Saturday night live used to be e...   
2                @jyy9624                   NY by way of 2nd city and Canada   
3  @justjakewolverine4619  &quot;RACIST&quot; from Kentucky here in &#39;...   
4             @Tonesnob72  Good to know if you don’t vote the way dems wa...   

   like_count              published_At  year  comment_length  video_year  
0           0 2025-08-

In [12]:
# Election Cycle berechnen
df_videos['published_at'] = pd.to_datetime(df_videos['published_at'])
df_videos['comments_per_view'] = df_videos['comment_Count'] / df_videos['view_Count']

# Funktion, um den Wahlzyklus basierend auf dem Jahr zu bestimmen
def determine_election_cycle(year):
    if 2014 <= year <= 2018:
        return 2016
    elif 2019 <= year <= 2022:
        return 2020
    elif year >= 2023:
        return 2024

# Wende die Funktion auf die 'video_year' Spalte an und erstelle die neue Spalte 'election_cycle'
df_videos['election_cycle'] = df_videos['video_year'].apply(determine_election_cycle)

# Ausgabe der ersten paar Zeilen zur Überprüfung
print(df_videos[['video_ID', 'video_year', 'election_cycle']].head())

      video_ID  video_year  election_cycle
0  SHG0ezLiVGc        2016            2016
1  Wsije1KetVw        2020            2020
2  g34CFjEFZts        2024            2024


In [13]:
# Election Cycle zu commets hinzufügen
df_comments = pd.merge(df_comments, df_videos[['video_ID', 'election_cycle']], on='video_ID', how='left')

print(df_comments.head())

   custom_index                  comment_ID parent_ID     video_ID  \
0             1  Ugze0aXA6zTFey241M94AaABAg       NaN  SHG0ezLiVGc   
1             2  Ugwa-qc1G5P9VZEjHuF4AaABAg       NaN  SHG0ezLiVGc   
2             3  UgxOS3FTiYFkiS9bGf14AaABAg       NaN  SHG0ezLiVGc   
3             4  UgyQQqHl9-Y13pPiml54AaABAg       NaN  SHG0ezLiVGc   
4             5  UgwBHGY9omjjiF2f1Gh4AaABAg       NaN  SHG0ezLiVGc   

                   author                                               text  \
0         @dennistims2604                           2025 and still laughing.   
1            @klvd43rs565  DECADES AGO-  Saturday night live used to be e...   
2                @jyy9624                   NY by way of 2nd city and Canada   
3  @justjakewolverine4619  &quot;RACIST&quot; from Kentucky here in &#39;...   
4             @Tonesnob72  Good to know if you don’t vote the way dems wa...   

   like_count              published_At  year  comment_length  video_year  \
0           0 2025-08

In [14]:
df_videos.to_csv('Export_csv/videos_v1.csv',  encoding='utf-8', quoting=csv.QUOTE_ALL)
df_comments.to_csv('Export_csv/comments_v1.csv',  encoding='utf-8', quoting=csv.QUOTE_ALL)

print('CSV files have been successfully saved.')

CSV files have been successfully saved.
